<a href="https://colab.research.google.com/github/locdeng/RotbotVisionProject/blob/main/ColorClassificationUsingResnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision matplotlib

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:

import os
import torchvision.transforms as transforms
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import random

#Define the ColorDataset class
class ColorDataset(Dataset):
    def __init__(self, base_dir, transform=None, train=True, split_ratio=0.8, seed=42):
        self.base_dir = base_dir
        self.transform = transform
        self.train = train
        self.split_ratio = split_ratio
        self.seed = seed
        self.classes = sorted(os.listdir(base_dir))
        self.num_classes = len(self.classes)

        # Create a list to store the paths of all images
        self.all_images = []
        for i, color_class in enumerate(self.classes):
            class_path = os.path.join(base_dir, color_class)
            images = [os.path.join(class_path, img) for img in os.listdir(class_path)]
            self.all_images.extend([(img, i) for img in images])  # (image path, class index)

        # Randomly shuffle the list of images
        random.seed(seed)
        random.shuffle(self.all_images)

        # Split the dataset into training and testing sets
        split_idx = int(len(self.all_images) * split_ratio)
        if self.train:
            self.images = self.all_images[:split_idx]
        else:
            self.images = self.all_images[split_idx:]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path, label = self.images[index]

        try:
            print(f"Loading image: {img_path}")
            image = Image.open(img_path).convert("RGB")

            if self.transform is not None:
                image = self.transform(image)

            return image, label
        except (OSError, PIL.UnidentifiedImageError) as e:
            print(f"Error loading image {img_path}: {e}")
            return None, None

#Define the transformations
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

#Create an instance of the ColorDataset
base_dir = '/content/gdrive/MyDrive/color'
color_dataset = ColorDataset(base_dir, transform=transform_train, train=True)

#Create a DataLoader
color_loader = DataLoader(color_dataset, batch_size=32, shuffle=True, num_workers=2)


In [ ]:
import torch
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
from torch import nn, optim
import cv2
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import time
from tqdm import tqdm


# Load pre-trained ResNet-18 and ResNet-50 models
resnet18 = models.resnet18(pretrained=True)
resnet50 = models.resnet50(pretrained=True)

# Set the models to evaluation mode
resnet18.eval()
resnet50.eval()

In [ ]:
import matplotlib.pyplot as plt

# Modify the last fully connected layer for multi-color classification
num_classes = len(color_dataset.classes)
resnet18.fc = nn.Linear(resnet18.fc.in_features, num_classes)
resnet50.fc = nn.Linear(resnet50.fc.in_features, num_classes)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = resnet18.to(device)
resnet50 = resnet50.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer_resnet18 = optim.Adam(resnet18.parameters(), lr=0.001)
optimizer_resnet50 = optim.Adam(resnet50.parameters(), lr=0.001)

def train_model(model, criterion, optimizer, dataloader, num_epochs=10):
    model.train()
    epoch_loss = []

    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in dataloader:
            if inputs is None or labels is None:
                continue

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        epoch_loss.append(running_loss / len(dataloader))
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss[-1]}')

    # Plotting training loss
    plt.plot(epoch_loss, label='Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss Over Epochs')
    plt.legend()
    plt.show()

    return epoch_loss

# Train the ResNet18 model
train_loss_resnet18 = train_model(resnet18, criterion, optimizer_resnet18, color_loader, num_epochs=10)
# Save the trained ResNet18 model
torch.save(resnet18.state_dict(), '/content/resnet18_color_classification.pth')
# Train the ResNet50 model
train_loss_resnet50 = train_model(resnet50, criterion, optimizer_resnet50, color_loader, num_epochs=10)
# Save the trained ResNet50 model
torch.save(resnet50.state_dict(), '/content/resnet50_color_classification.pth')

In [ ]:
plt.plot(train_loss_resnet18, label='ResNet18 Training Loss')
plt.plot(train_loss_resnet50, label='ResNet50 Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Comparison Between ResNet18 and ResNet50')
plt.legend()
plt.show()

In [ ]:


# Evaluating
def evaluate_model(model, dataloader):
    model.eval()
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Evaluating"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predictions = torch.max(outputs, 1)

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_predictions)
    classification_report_str = classification_report(all_labels, all_predictions)
    confusion_mat = confusion_matrix(all_labels, all_predictions)

    return accuracy, classification_report_str, confusion_mat

# Evaluate the ResNet18 model
accuracy_resnet18, report_resnet18, confusion_mat_resnet18 = evaluate_model(resnet18, color_loader)
print(f"ResNet18 Accuracy: {accuracy_resnet18:.4f}")
print("Classification Report (ResNet18):")
print(report_resnet18)
print("Confusion Matrix (ResNet18):")
print(confusion_mat_resnet18)

# Evaluate the ResNet50 model
accuracy_resnet50, report_resnet50, confusion_mat_resnet50 = evaluate_model(resnet50, color_loader)
print(f"\nResNet50 Accuracy: {accuracy_resnet50:.4f}")
print("Classification Report (ResNet50):")
print(report_resnet50)
print("Confusion Matrix (ResNet50):")
print(confusion_mat_resnet50)

# Compare the results
if accuracy_resnet18 > accuracy_resnet50:
    print("\nResNet18 performs better.")
elif accuracy_resnet18 < accuracy_resnet50:
    print("\nResNet50 performs better.")
else:
    print("\nBoth models have similar performance.")




In [ ]:
import matplotlib.pyplot as plt


# Visualize the results in a bar chart
labels = ['ResNet18', 'ResNet50']
accuracies = [accuracy_resnet18, accuracy_resnet50]

plt.bar(labels, accuracies, color=['blue', 'orange'])
plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.title('Comparison of ResNet18 and ResNet50 Accuracy')
plt.ylim(0, 1)
plt.show()

In [ ]:
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv2_imshow

# Create ResNet18 model
resnet50 = models.resnet50(pretrained=True)
# Modify the final fully connected layer for color classification
resnet50.fc = nn.Linear(resnet50.fc.in_features, num_classes)

# Load the trained weights
checkpoint_path = '/content/resnet50_color_classification.pth'
# checkpoint_path = '/content/resnet18_color_classification.pth'
resnet50.load_state_dict(torch.load(checkpoint_path))
resnet50 = resnet18.to(device)
resnet50.eval()

# Preprocess the input image
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])


    return input_image

image_path = # test image result
input_image = preprocess_image(image_path)

# Pass the input through the model
with torch.no_grad():
    output = resnet18(input_image)

# Get the predicted class index
_, predicted_class_idx = torch.max(output, 1)

# Map the predicted class index to the corresponding color class
predicted_color = color_dataset.classes[predicted_class_idx.item()]

# Print information
image =cv2.imread('image_path')
cv2_imshow(image)
print(f"Predicted color: {predicted_color}")